In [ ]:
import cv2
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import random
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self, num_classes=2, kernel_size=3, max_pool_kernel_size=2):
        super(ConvolutionalNeuralNetwork, self).__init__()
        
        self.num_classes = num_classes
        self.kernel_size = kernel_size
        self.max_pool_kernel_size = max_pool_kernel_size
        
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=self.kernel_size)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=self.kernel_size)
        
        self.max_pool1 = nn.MaxPool2d(kernel_size = self.max_pool_kernel_size, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=self.kernel_size)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=self.kernel_size)
        
        self.max_pool2 = nn.MaxPool2d(kernel_size = self.max_pool_kernel_size, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, self.num_classes)
        
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        
        out = self.max_pool2(out)
        
        # Flatten things out
        # out = out.reshape(out.shape(0), -1)
        out = out.view(-1, 1600)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        
        return out

In [ ]:
class CnnDataset(Dataset):
    def __init__(self, img_dir, img_width, img_height, label):
        self.img_dir    = img_dir
        self.img_width  = img_width
        self.img_height = img_height
        self.images     = os.listdir(img_dir)
        self.label      = label

        self.dim = (img_width, img_height)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.img_dir, self.images[index])

        img = (cv2.resize(cv2.imread(img_path), self.dim) / 255).transpose((2, 0, 1)) 

        # Input is the Output
        output = torch.Tensor(img), torch.Tensor(self.label)

        return output

In [ ]:
def train_fn(loader, model, optimizer, loss_fn):
    loop = tqdm(loader)

    ave_loss = 0.0
    count = 0

    for batch_idx, (data, targets) in enumerate(loop):
        data    = data.float().to(device=device)
        targets = targets.float().to(device=device)

        # Forward
        predictions = model.forward(data)

        loss = loss_fn(predictions, targets)

        # Backward
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()

        # update tqdm
        loop.set_postfix(loss=loss.item())

        ave_loss += loss.item()
        count += 1

    ave_loss = ave_loss / count

    return ave_loss

In [ ]:
device         = 'cuda'
gpu_index      = 0
epochs         = 10
learning_rate  = 0.0001
chunk_size     = 1
batch_size     = 1
cont           = True
model_file     = "cnn-model.pth"
padding        = 1
scale          = 2
img_width      = 32
img_height     = 32
train_img_dir  = "/home/ralampay/Desktop/Female"
label          = [1,0]
num_channels   = 3

# CNN Parameters
num_classes          = 2
kernel_size          = 3
max_pool_kernel_size = 2

In [ ]:
if device == 'cuda':
    print("CUDA Device: {}".format(torch.cuda.get_device_name(gpu_index)))
    device = "cuda:{}".format(gpu_index)

model = ConvolutionalNeuralNetwork(
    num_classes=num_classes, 
    kernel_size=kernel_size, 
    max_pool_kernel_size=max_pool_kernel_size
).to(device)

if cont:
    print("Loading model from {}".format(model_file))
    state = torch.load(model_file)
    model.load_state_dict(state['state_dict'])
    model.optimizer = state['optimizer']

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_ds = CnnDataset(
    img_dir=train_img_dir,
    img_width=img_width,
    img_height=img_height,
    label=label
)

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

In [ ]:
losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    ave_loss = train_fn(train_loader, model, optimizer, loss_fn)

    print("Ave Loss: {}".format(ave_loss))
    losses.append(ave_loss)

    # Save model after every epoch
    print("Saving model to {}...".format(model_file))

    state = {
        'state_dict':   model.state_dict(),
        'optimizer':    optimizer.state_dict()
    }

    torch.save(state, model_file)

    print("Done.")